In [1]:
import pandas as pd
from sodapy import Socrata
import os
from modules import data_management as dm
import glob
import zipfile
%load_ext autoreload
%autoreload 2

# Descargar archivos fuente en la carpeta 'local'

In [2]:
dm.download_rutinarias()

In [3]:
dm.download_file_maps()
dm.download_file_population()
dm.download_file_year()
dm.download_file_departments()

1.4 MiB B Done.
640.0 KiB Done.
448.0 KiB Done.
32.0 KiB Done.


In [4]:
dm.download_file_sstoi()
dm.download_file_tni()
dm.download_file_oni()
dm.download_file_meiv2()
dm.download_file_car()
dm.download_file_nta()

# Descarga paso a paso de los datos...

## Establecer la ruta en la que se guardarán los datos

Por defecto, se almacenan en la carpeta del repositorio. De darse algún error, se guardan en la ruta relativa ```./local/data```.

In [5]:
setup = "repo" #alternativo: local

try:
    if setup == "repo":
        data_folder = "/".join(dm.notebook_path().replace("\\","/").split("/")[:-1]) + "/local/data/"
    else:
        data_folder = "/".join(dm.notebook_path().replace("\\","/").split("/")[:-1])  + "/data/"
        
except:
    print("Couldn't assess path automatically, setting up path locally...")
    data_folder = "./local/data/"
    
dm.make_folder_if_not_exists(data_folder)
data_folder

Couldn't assess path automatically, setting up path locally...


'./local/data/'

## Escoger la ciudad y recuperar los datos relevantes del archivo ```params.csv```

In [6]:
df_cities = dm.read_cities_file()
df_cities

,department,municipality,code,lat,lon,dl_folder,starting_date,stations_temp,stations_hum,stations_prec,stations_rad,range_temp,range_hum
0,Antioquia,Medellín,05001,6.242667,-75.589640,MEDELLIN_DATA,2009-04-01,"[27015330, meteostat, 27015300]","[27015330, meteostat, 27015300]","[27015330, 27015290, 27011270]",[],"[15.5, 37.5]","[15, 99.99]"
1,Magdalena,Santa Marta,47001,NaN,NaN,SANTA_MARTA_DATA,2008-06-28,[15015120],[15015120],"[15015050, 15010501, 15015120]",[],"[17.5, 40]","[40, 99.99]"
2,Santander,Bucaramanga,68001,NaN,NaN,BUCARAMANGA_DATA,2014-12-11,[23195230],[23195230],[23195230],[],"[17, 40]","[30, 99.99]"
3,Atlántico,Barranquilla,08001,NaN,NaN,SOLEDAD_BARRANQUILLA_DATA,2013-05-03,"[29045190, 29004520]",[29045190],"[29045120, 29040450, 29045190]",[],"[22, 45]","[45, 99.99]"
4,Atlántico,Soledad,08758,NaN,NaN,SOLEDAD_BARRANQUILLA_DATA,2007-01-01,[],[],[],[],[],[]
5,Norte de Santander,Cúcuta,54001,7.904338,-72.511459,CUCUTA_DATA,2006-09-03,"[16015110, 16015501, meteostat]","[16015110, 16015501, meteostat]","[meteostat, 16015110, 16015010, 16015501]",[],"[17.5, 42.5]","[30, 99.99]"
6,Valle del Cauca,Cali,76001,3.413628,-76.517484,CALI_DATA,2006-11-24,"[26085170, meteostat, 26055120]","[26085170, meteostat, 26055120]","[26055120, 26080310, 26085170]",[],"[16.5, 45]","[30, 99.99]"
7,Valle del Cauca,Palmira,76563,NaN,NaN,CALI_DATA,2006-11-24,[],[],[],[],[],[]
8,Caquetá,Villavicencio,50001,NaN,NaN,VILLAVICENCIO_DATA,2007-03-18,"[35035100, 35025110]","[35035100, 35025110]","[35030380, 35030370, 35030100, 35035020]",[],"[20, 45]","[35, 99.99]"
9,Tolima,Ibagué,73001,NaN,NaN,IBAGUE_DATA,2007-01-01,[],[],[],[],[],[]


In [53]:
# datos listos 2021: MEDELLIN, SANTA MARTA, LETICIA, PUERTO NARIÑO, CUCUTA
row_id = 6
params = {}
params_to_get = ["department","municipality","dl_folder"]
for param in params_to_get:
    params[param] = df_cities.loc[row_id,param]
params

{'department': 'Valle del Cauca',
 'municipality': 'Cali',
 'dl_folder': 'CALI_DATA'}

In [54]:
dl_folder = data_folder + "src_per_city/" + params["dl_folder"]+"/"
#dl_folder = dl_folder.replace("/", "\\")
dm.make_folder_if_not_exists(dl_folder)
dl_folder

'./local/data/src_per_city/CALI_DATA/'

## Descargar datos del mapa de IDEAM usando Selenium

El API del IDEAM solo permite descargas de estas variables en tandas de a 5 años consecutivos, así que dividimos los datos en segmentos de 5 años. 

Nuestro segmento relevante es 2005 a 2022, pero se pueden descargar más segmentos de ser necesario.

In [55]:
variables = ["PRECIPITACION","HUM RELATIVA","TEMPERATURA",] #"RAD SOLAR"]
date_intervals = [['01/01/2005','31/12/2009'],
                    ['01/01/2010','31/12/2014'],
                    ['01/01/2015','31/12/2019'],
                    ['01/01/2020','31/12/2022']
                ]
date_intervals

[['01/01/2005', '31/12/2009'],
 ['01/01/2010', '31/12/2014'],
 ['01/01/2015', '31/12/2019'],
 ['01/01/2020', '31/12/2022']]

In [56]:
for interval in date_intervals:
    for variable in variables:
        res = dm.download_data_with_selenium(params["department"], params["municipality"], variable, interval, dl_folder)
        if res == True:
            print("Started download for:",params["department"],"/",params["municipality"],"/",variable,interval)
            try:
                os.rename(dl_folder+"datos.zip",dl_folder+"{}.zip".format(params["department"]+"-"+params["municipality"]+"-"+variable+"-"+interval[0].replace("/","-")+"-"+interval[1].replace("/","-")))
            except Exception as e:
                print(e)
                print("Error downloading, driver closed prematurely.")
        else:
            print("Couldn't find data for:",params["department"],"/",params["municipality"],"/",variable,interval)

Retrying get modal
Retrying get modal
Retrying get modal
Retrying get modal
Retrying get datepickers
Started download for: Valle del Cauca / Cali / PRECIPITACION ['01/01/2005', '31/12/2009']
Retrying get modal
Retrying get modal
Retrying get modal
Retrying get modal
Retrying get datepickers
Started download for: Valle del Cauca / Cali / HUM RELATIVA ['01/01/2005', '31/12/2009']
Retrying get modal
Retrying get modal
Retrying get modal
Retrying get modal
Retrying get datepickers
Started download for: Valle del Cauca / Cali / TEMPERATURA ['01/01/2005', '31/12/2009']
Retrying get modal
Retrying get modal
Retrying get modal
Retrying get modal
Retrying get datepickers
Started download for: Valle del Cauca / Cali / PRECIPITACION ['01/01/2010', '31/12/2014']
Retrying get modal
Retrying get modal
Retrying get modal
Retrying get modal
Retrying get datepickers
Started download for: Valle del Cauca / Cali / HUM RELATIVA ['01/01/2010', '31/12/2014']
Retrying get modal
Retrying get modal
Retrying ge

In [57]:
zipped_csvs = glob.glob(dl_folder+"*.zip")
files_count = 0
for zipped_csv in zipped_csvs:
    filename = zipped_csv.split("/")[-1].split(".")[0]
    zipdata = zipfile.ZipFile(zipped_csv)
    zipdata.extract("excel.csv.csv", path = dl_folder)
    try:
        os.rename(dl_folder+"excel.csv.csv",dl_folder+"{}.csv".format(filename))
    except:
        print("File exists already")
    files_count+=1
    zipdata.close()

# DOWNLOAD FROM DATOS ABIERTOS API


In [58]:
# Example authenticated client (needed for non-public datasets):
client = Socrata('www.datos.gov.co', "mc0EDr4vP5YMKFeiaAEHZUWX0")
Precipitacion = 's54a-sgyg'
Humedad = 'uext-mhny'
Temperatura = 'sbwg-7ju4'
Viento = 'sgfv-3yp8'

# PRECIPITATION DATA DOWNLOAD

In [60]:
prec_stations = dm.get_stations_from_municipality(client, Precipitacion, params["municipality"].upper())
prec_stations

HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying


,CodigoEstacion,CodigoSensor,NombreEstacion,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0026085160,0240,SILOE - AUT,3.4255,-76.56061111,Precipitacion,mm
1,0026055100,0240,FARALLONES - AUT,3.416055556,-76.6515,Precipitacion,mm
2,0026055120,0240,UNIVERSIDAD DEL VALLE - AUT,3.378,-76.53388889,Precipitacion,mm
3,0026085170,0240,BASE AEREA MARCO FIDEL SUAREZ - AUT,3.4545,-76.49972222,Precipitacion,mm
4,2629700151,0240,LA VORAGINE - AUT,3.346305556,-76.589333333,Precipitacion,mm


In [61]:
codigos_estaciones = prec_stations["CodigoEstacion"].values
prec_time_series = []
for codigo in codigos_estaciones:
    precipitation_base_data = dm.get_time_series_from_station(client, Precipitacion,  params["municipality"].upper(), codigoestacion=codigo, limit = 1000000)
    precipitation_data = dm.group_variable(precipitation_base_data,'daily')
    prec_time_series.append(precipitation_data)
all_prec_series = pd.concat(prec_time_series,axis = 0).reset_index(drop=True)
all_prec_series = dm.API_to_IDEAM_series(all_prec_series)
all_prec_series.head(2)

,CodigoEstacion,NombreEstacion,Latitud,Longitud,Altitud,Categoria,Entidad,AreaOperativa,Departamento,Municipio,...,FechaSuspension,IdParametro,Etiqueta,DescripcionSerie,Frecuencia,Fecha,Valor,Grado,Calificador,NivelAprobacion
0,26085160,SILOE - AUT,3.4255,-76.56061111,,mm,,CAUCA,VALLE DEL CAUCA,CALI,...,,,Precipitacion,,,2016-12-31T00:00:00.000,0.0,,,
1,26085160,SILOE - AUT,3.4255,-76.56061111,,mm,,CAUCA,VALLE DEL CAUCA,CALI,...,,,Precipitacion,,,2017-01-01T00:00:00.000,0.0,,,


In [62]:
all_prec_series.to_csv(dl_folder+"zAPI_{}_precipitacion.csv".format(params["municipality"]), index = False)

# TEMPERATURE DATA DOWNLOAD

In [63]:
temp_stations = dm.get_stations_from_municipality(client, Temperatura, params["municipality"].upper())
temp_stations

,CodigoEstacion,CodigoSensor,NombreEstacion,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0026055120,0068,UNIVERSIDAD DEL VALLE - AUT,3.378,-76.53388889,Temp Aire 2 m,°C
1,0026085160,0068,SILOE - AUT,3.4255,-76.56061111,Temp Aire 2 m,°C
2,0026055100,0068,FARALLONES - AUT,3.416055556,-76.6515,Temp Aire 2 m,°C
3,0026085170,0068,BASE AEREA MARCO FIDEL SUAREZ - AUT,3.4545,-76.49972222,Temp Aire 2 m,°C


In [64]:
codigos_estaciones = temp_stations["CodigoEstacion"].values
temp_time_series = []
for codigo in codigos_estaciones:
    temperature_data = dm.group_variable(dm.get_time_series_from_station(client, Temperatura,  params["municipality"].upper(), codigoestacion=codigo, limit = 1000000), 'hourly')
    temp_time_series.append(temperature_data)
all_temp_series = pd.concat(temp_time_series,axis = 0).reset_index(drop=True)
all_temp_series = dm.API_to_IDEAM_series(all_temp_series)
all_temp_series.to_csv(dl_folder+"/zAPI_{}_temperature.csv".format(params["municipality"]), index = False)

# HUMEDAD DATA DOWNLOAD

In [65]:
hum_stations = dm.get_stations_from_municipality(client, Humedad, params["municipality"].upper())
hum_stations

HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying


,CodigoEstacion,CodigoSensor,NombreEstacion,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0026085160,0027,SILOE - AUT,3.4255,-76.56061111,Humedad del aire 2 mt,%
1,0026085170,0027,BASE AEREA MARCO FIDEL SUAREZ - AUT,3.4545,-76.49972222,Humedad del aire 2 mt,%
2,0026055100,0027,FARALLONES - AUT,3.416055556,-76.6515,Humedad del aire 2 mt,%
3,0026055120,0027,UNIVERSIDAD DEL VALLE - AUT,3.378,-76.53388889,Humedad del aire 2 mt,%


In [66]:
codigos_estaciones = hum_stations["CodigoEstacion"].values
hum_time_series = []
for codigo in codigos_estaciones:
    humedad_data = dm.group_variable(dm.get_time_series_from_station(client, Humedad,  params["municipality"].upper(), codigoestacion=codigo, limit = 1000000))
    hum_time_series.append(humedad_data)
hum_time_series = pd.concat(hum_time_series,axis = 0).reset_index(drop=True)
hum_time_series = dm.API_to_IDEAM_series(hum_time_series)
hum_time_series.to_csv(dl_folder+"/zAPI_{}_humidity.csv".format(params["municipality"]), index = False)

# WIND DATA DOWNLOAD

In [67]:
win_stations = dm.get_stations_from_municipality(client, Viento, params["municipality"].upper())
win_stations

HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConne

,CodigoEstacion,CodigoSensor,NombreEstacion,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0026085170,0103,BASE AEREA MARCO FIDEL SUAREZ - AUT,3.4545,-76.49972222,Velocidad Viento (10 min),m/s
1,0026085160,0103,SILOE - AUT,3.4255,-76.56061111,Velocidad Viento (10 min),m/s
2,0026055120,0103,UNIVERSIDAD DEL VALLE - AUT,3.378,-76.53388889,Velocidad Viento (10 min),m/s
3,0026055100,0103,FARALLONES - AUT,3.416055556,-76.6515,Velocidad Viento (10 min),m/s


In [33]:
codigos_estaciones = win_stations["CodigoEstacion"].values
win_time_series = []
for codigo in codigos_estaciones:
    wind_data = dm.group_variable(dm.get_time_series_from_station(client, Viento,  params["municipality"].upper(), codigoestacion=codigo, limit = 1000000))
    win_time_series.append(wind_data)
win_time_series = pd.concat(win_time_series,axis = 0).reset_index(drop=True)
win_time_series = dm.API_to_IDEAM_series(win_time_series)
win_time_series.to_csv(dl_folder+"/zAPI_{}_wind.csv".format(params["municipality"]), index = False)